In [4]:
import os

In [5]:
DATA_PATH_JSON = "./NLP-Teknofest24/NLPJSON/"
DATA_PATH_TXT = "./NLP-Teknofest24/NLPTXT-1028 Etiketlenen Raporlar/"

In [6]:
import json
d_list = list()
for i in range(1,5):
    with open(DATA_PATH_JSON+str(i)+"/all.jsonl",'r', encoding="utf-8") as f:
        for a in f.readlines():
            d_list.append(json.loads(a))

In [7]:
import pandas as pd

In [8]:
meta_data = pd.DataFrame(d_list)

In [9]:
meta_data = meta_data.drop("Comments",axis=1)

In [10]:
meta_data = meta_data.sort_values("id")

In [11]:
data_df = meta_data.drop("id",axis=1)

In [12]:
data_df = data_df.reset_index().drop("index",axis=1)

In [13]:
data_df

,text,label
0,BİLATERAL MAMOGRAFİ İNCELEMESİ\nMeme parankimi...,"[[31, 45, ANAT], [46, 59, OBS-PRESENT], [75, 8..."
1,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme p...,"[[31, 43, ANAT], [44, 54, ANAT], [58, 76, OBS-..."
2,BİLATERAL MAMOGRAFİ İNCELEMESİ\nMemeler dağını...,"[[31, 38, ANAT], [39, 46, ANAT], [47, 79, OBS-..."
3,BİLATERAL MAMOGRAFİ İNCELEMESİ\nMemeler ACR Ti...,"[[31, 38, ANAT], [39, 48, OBS-PRESENT], [50, 7..."
4,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme t...,"[[31, 43, ANAT], [45, 62, OBS-PRESENT], [63, 7..."
...,...,...
1023,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSol meme para...,"[[32, 40, ANAT], [41, 58, ANAT], [58, 89, OBS-..."
1024,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSol meme cilt...,"[[32, 40, ANAT], [40, 55, ANAT], [56, 65, OBS-..."
1025,UNİLATERAL MAMOGRAFİ RAPORU\nSol meme dokusu t...,"[[28, 44, ANAT], [44, 49, OBS-PRESENT], [49, 7..."
1026,UNİLATERAL MAMOGRAFİ İNCELEMESİ:\nSağ meme pat...,"[[33, 49, ANAT], [50, 63, OBS-PRESENT], [64, 7..."


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# First split: Train+Validation and Test
train_val_df, test_df = train_test_split(data_df, test_size=250, random_state=42)

# Second split: Train and Validation
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

In [22]:
import string

def clean_annotations(df):
    cleaned_entities = []
    for _, row in df.iterrows():
        text = row['text']
        entities = []
        for start, end, label in row['label']:
            # Strip leading/trailing whitespace or punctuation from the entity
            while text[start] in string.whitespace + string.punctuation:
                start += 1
            while text[end - 1] in string.whitespace + string.punctuation:
                end -= 1
            entities.append((start, end, label))
        cleaned_entities.append(entities)
    df['label'] = cleaned_entities
    return df

train_df = clean_annotations(train_df)


In [21]:
import re

def remove_redundant_spaces_regex(text: str) -> str:
  return re.sub(r"\s+", " ", text)

In [23]:
TRAIN_DATA = [(row['text'], {'entities': row['label']}) for _, row in train_df.iterrows()]

In [24]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding

# Load a blank Turkish model
nlp = spacy.blank("tr")

# Create the NER component and add it to the pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Add the labels to the NER component
LABELS = ["ANAT", "OBS-PRESENT", "OBS-ABSENT", "OBS_UNCERTAIN"]
for label in LABELS:
    ner.add_label(label)

# Disable other pipelines during training
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Start training
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(30):  # 30 iterations
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            examples = [Example.from_dict(nlp.make_doc(text), annot) for text, annot in zip(texts, annotations)]
            nlp.update(examples, drop=0.5, losses=losses)
        print(f"Iteration {itn} - Losses: {losses}")

# Save the model
nlp.to_disk("custom_ner_model_tr")

# Load the saved model
nlp = spacy.load("custom_ner_model_tr")

# Test the model
doc = nlp("Hasta femurunda kırık var")
for ent in doc.ents:
    print(ent.text, ent.label_)


d:\Anaconda\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "BİLATERAL MAMOGRAFİ İNCELEMESİ:
Bilateral areola, ..." with entities "[(32, 48, 'ANAT'), (50, 54, 'ANAT'), (56, 65, 'ANA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Anaconda\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "BİLATERAL MAMOGRAFİ İNCELEMESİ
Memeler ACR Tip C  ..." with entities "[(31, 38, 'ANAT'), (44, 58, 'OBS-PRESENT'), (65, 8...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Anaconda\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "BİLATERAL

Iteration 0 - Losses: {'ner': 20761.062325260846}


ValueError: [E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help